In [65]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1
#!pip install onnxruntime==1.21.1

In [2]:
import os

move_filename = '0cced52'
moves_filepath = os.path.join('outputs', 'moves.json')

In [4]:
!ffmpeg -i inputs/videos/0cced52.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/videos/0cced52.wav

ffmpeg version 2025-04-23-git-25b0a8e295-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev3, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx

In [6]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', 'videos', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

C:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


array([[0.22, 1.  ],
       [0.58, 2.  ],
       [0.9 , 1.  ],
       [1.26, 2.  ],
       [1.58, 1.  ],
       [1.92, 2.  ],
       [2.28, 1.  ],
       [2.6 , 2.  ],
       [2.96, 1.  ],
       [3.3 , 2.  ],
       [3.64, 1.  ],
       [3.98, 2.  ],
       [4.3 , 1.  ],
       [4.66, 2.  ],
       [5.  , 1.  ],
       [5.34, 2.  ],
       [5.68, 1.  ],
       [6.02, 2.  ],
       [6.34, 1.  ],
       [6.68, 2.  ]])

In [26]:
''' '''
beat_start = 0.22
beat_end = 6.68
num_beats = 16
beat_delta = (beat_end - beat_start) / num_beats
beats = [[round(beat_start + x * beat_delta, 2), x % 8] for x in range(num_beats)]
beats


[[0.22, 0],
 [0.62, 1],
 [1.03, 2],
 [1.43, 3],
 [1.83, 4],
 [2.24, 5],
 [2.64, 6],
 [3.05, 7],
 [3.45, 0],
 [3.85, 1],
 [4.26, 2],
 [4.66, 3],
 [5.06, 4],
 [5.47, 5],
 [5.87, 6],
 [6.28, 7]]

In [28]:
# 9da1eb8 = 4
# 5db13cd = 1
# 5d80c9a = 1
offset = 0

In [30]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/videos/{move_filename}.mp4"
output_file = f"processed/videos/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = True
text_clips = [video_clip]

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{(index - offset) % 8 + 1}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)

        text_clips.append(text_clip)
        
# Composite the text clip on the video
video_clip = CompositeVideoClip(text_clips)

# Write the output video file
video_clip.write_videofile(output_file, fps=30, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [360, 640], 'bitrate': 1484, 'fps': 30.15, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 6.72, 'bitrate': 1619, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 127, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_codec_na

MoviePy - Done.
MoviePy - Writing video processed/videos/0cced52_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/videos/0cced52_counts.mp4


In [32]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
section = 1
for index, x in enumerate(beats):
    count = (index - offset) % 8 + 1
    print(count)
    data[move_filename].append({
        'section': section, 
        'count': count,
        'time': x[0],
        'leader_keypoints': [],
        'follower_keypoints': []
    })
    if count == 8:
        section += 1

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8


In [34]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = (index - offset) % 8 + 1
    print(count)
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file

    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8:
        section += 1

1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8


In [40]:
import numpy as np
import cv2
import os
from rtmlib import Body, draw_skeleton
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

device = 'cpu'
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

cap = cv2.VideoCapture(0)

openpose_skeleton = False  # True for openpose-style, False for mmpose-style

body = Body(
    pose='rtmo',
    to_openpose=openpose_skeleton,
    mode='performance',  # balanced, performance, lightweight
    backend=backend,
    device=device
)
# TODO: Nowadays, I do this by hand. We need to automate it.
leader_detected_first = [
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    False,
    False,
    False,
    True,
    False,
    True
]
#leader_detected_first = [False for index in range(len(beats))]
img_index = 0

for filename in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (filename.startswith(move_filename)):
        frame = cv2.imread(os.path.join('processed', 'images', filename), cv2.IMREAD_COLOR)
        keypoints, scores = body(frame)

        if leader_detected_first[img_index]:
            data[move_filename][img_index]["leader_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["follower_keypoints"] = keypoints[1].tolist()
        elif np.sum(keypoints) > 0:
            data[move_filename][img_index]["follower_keypoints"] = keypoints[0].tolist()
            if keypoints.shape[0] > 1:
                data[move_filename][img_index]["leader_keypoints"] = keypoints[1].tolist()
        print(filename)
        print(keypoints.shape)
        print(np.sum(keypoints))
        print(keypoints[0])

        img_show = frame.copy()
        img_show = draw_skeleton(
            img_show,
            keypoints,
            #np.array([keypoints[0]]),
            scores,
            openpose_skeleton=openpose_skeleton,
            kpt_thr=0.3,
            line_width=2
        )

        cv2.imwrite(os.path.join('processed', 'keypoints', filename), img_show)
        img_index += 1

# Override the data with the keypoints
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

load C:\Users\user\.cache\rtmlib\hub\checkpoints\rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.onnx with onnxruntime backend
0cced52_section_1_count_1.jpg
(1, 17, 2)
8403.424
[[136.24968  273.34686 ]
 [142.84581  270.14697 ]
 [132.4108   268.5153  ]
 [150.76239  280.2353  ]
 [125.54301  276.87173 ]
 [164.90298  302.1903  ]
 [117.22967  316.7577  ]
 [199.74399  317.1835  ]
 [ 94.67469  339.1796  ]
 [230.54767  321.55933 ]
 [ 74.53734  354.7995  ]
 [155.63779  385.33496 ]
 [124.211685 385.1642  ]
 [189.07407  431.69727 ]
 [125.96265  445.9539  ]
 [171.17038  491.94638 ]
 [110.777    496.25952 ]]
0cced52_section_1_count_2.jpg
(2, 17, 2)
17736.168
[[154.57108 265.15503]
 [152.66406 260.84155]
 [151.4716  260.45166]
 [126.76014 262.77716]
 [139.14082 263.22455]
 [119.33899 292.51764]
 [146.4064  289.65982]
 [138.04036 320.94724]
 [180.96982 315.83817]
 [158.83324 322.63217]
 [169.66162 294.89835]
 [117.82728 366.53195]
 [135.38875 367.08826]
 [140.29962 436.72272]
 [151.54918 436.35986

In [42]:
import numpy as np

def normalize_keypoints_by_bbox_height(keypoints):
    """
    Normalize keypoints so that:
    - The top-left of the bounding box is (0, 0)
    - The bounding box height is 1
    """
    kp = np.array(keypoints)
    
    min_x = np.min(kp[:, 0])
    min_y = np.min(kp[:, 1])
    max_y = np.max(kp[:, 1])
    
    bbox_height = max_y - min_y
    if bbox_height < 1e-5:
        raise ValueError("Bounding box height too small for normalization.")
    
    # Translate to origin and scale
    kp_normalized = (kp - [min_x, min_y]) / bbox_height
    return kp_normalized, bbox_height


def compute_normalized_embeddings(leader_kp, follower_kp):
    leader_kp = np.array(leader_kp)
    follower_kp = np.array(follower_kp)

    # Normalize each person's keypoints by their own bbox height
    norm_leader = np.array([])
    norm_follower = np.array([])
    if np.sum(leader_kp) > 0:
        norm_leader, leader_height = normalize_keypoints_by_bbox_height(leader_kp)
    if np.sum(follower_kp) > 0:
        norm_follower, _ = normalize_keypoints_by_bbox_height(follower_kp)

    rel_vector = np.array([])
    if np.sum(leader_kp) > 0 and np.sum(follower_kp) > 0:
        # Get original centers (before normalization)
        center_leader = np.mean(leader_kp, axis=0)
        center_follower = np.mean(follower_kp, axis=0)
    
        # Compute relative position vector in leader's height units
        rel_vector = (center_follower - center_leader) / leader_height

    return norm_leader.flatten(), norm_follower.flatten(), rel_vector

In [44]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

for index in range(len(data[move_filename])):
    kp_leader = data[move_filename][index]['leader_keypoints']
    kp_follower = data[move_filename][index]['follower_keypoints']
    leader_embedding, follower_embedding, distance_embedding = compute_normalized_embeddings(kp_leader, kp_follower)
    data[move_filename][index]['leader_embedding'] = leader_embedding.tolist()
    data[move_filename][index]['follower_embedding'] = follower_embedding.tolist()
    data[move_filename][index]['distance_embedding'] = distance_embedding.tolist()

# Override the data with the new embeddings
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [63]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# Add musicality (TODO: This is manual)
musicality = [
    '-',
    '-',
    '-',
    '-',
    '-',
    '-',
    '*',
    '-',
    '-'
]
for index in range(len(data[move_filename])):
    data[move_filename][index]['musicality'] = musicality[index]

# Override the data with musicality
with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)